Importing the libraries

In [1]:
import sklearn
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn import metrics
from sklearn import preprocessing
import numpy as np

Read data file

In [2]:
df = pd.read_csv("out_without_nan_mood_normalised.csv")

Add dummies, remove weekDay mean, remove day mean

In [3]:
df['mood_mean_TARGET'] = df['mood_mean_TARGET'].astype(float)
df = pd.concat([df,pd.get_dummies(df['id'])],axis=1)
df = pd.concat([df,pd.get_dummies(df['weekDay'],prefix="day_")],axis=1)
df = df.drop(['weekDay_time_5','day_time_5','weekDay_time_4','day_time_4','weekDay_time_3','day_time_3','weekDay_time_2','day_time_2','weekDay_time_1','day_time_1','weekDay_time_5'],axis=1)

Split in train and test set

In [24]:
corr=df.corr()
goodColumns= corr[abs(corr['mood_mean_TARGET'])>0.4]['mood_mean_TARGET'].sort_values()
goodColumns = goodColumns.drop(['mood_mean_TARGET','mood_mean_time_1'])
print(goodColumns)


mood_mean           0.472047
mood_mean_time_3    0.483793
mood_mean_time_4    0.494474
mood_mean_time_2    0.499446
mood_mean_time_5    0.504306
Name: mood_mean_TARGET, dtype: float64


In [25]:
notRelevantAll=df.drop(['id','date'],axis=1).dropna()
goldY= notRelevantAll['mood_mean_TARGET']
relevant = notRelevantAll.drop(['mood_mean_TARGET'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(relevant[goodColumns.keys()], goldY, test_size=0.33, random_state=42)

In [26]:
clf = sklearn.svm.SVR(gamma="scale")
clf = clf.fit(X_train, y_train)

Define scoring

In [27]:
scoring = {'R2': 'r2',
           'Mean squared Error': 'neg_mean_squared_error',
           'Explained variance': 'explained_variance'}

In [28]:
y_pred = clf.predict(X_test)
scores = cross_validate(clf, relevant[goodColumns.keys()], goldY, cv=7,scoring=scoring)

print("R2: %0.2f (+/- %0.2f)" % (scores['test_R2'].mean(), scores['test_R2'].std() * 2))
print("MSE: %0.2f (+/- %0.2f)" % (scores['test_Mean squared Error'].mean(), scores['test_Mean squared Error'].std() * 2))
print("EXPLAINED VARIANCE: %0.2f (+/- %0.2f)" % (scores['test_Explained variance'].mean(), scores['test_Explained variance'].std() * 2))


R2: 0.19 (+/- 0.27)
MSE: -0.40 (+/- 0.26)
EXPLAINED VARIANCE: 0.20 (+/- 0.29)


In [22]:
y_pred = clf.predict(X_test)
print("MAE")
print(metrics.mean_absolute_error(y_test,y_pred))
print("MSE")
print(metrics.mean_squared_error(y_test,y_pred))
print("RMSE")
print(np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
print("R2")
print(metrics.r2_score(y_test,y_pred))

MAE
0.45750698455899863
MSE
0.3942013273507141
RMSE
0.6278545431473074
R2
0.22656829187925032
